In [ ]:
import pandas as pd
import requests
import sqlalchemy
import pymysql
import bs4
from bs4 import BeautifulSoup as bs
import unicodedata
import re
import numpy as np

#Get flight infos
def get_flight_info(flight_json, airport_icoa):
# terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
# aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code':airport_icoa
        }

#Get population data 
cities = ['Berlin','Paris','Amsterdam','Barcelona','Rome','Lisbon','Prague','Vienna','Madrid', 'Stuttgart']
def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    
    
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)
    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()
    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()
    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
        
    return ret_dict
    
#Start Lambda function
def lambda_handler(event, context):
#Get flight information
    airport_icoa = "EDDS"
    to_local_time = "2021-12-01T20:00"
    from_local_time = "2021-12-02T08:00"
    flight_api_key = "e39db7f26bmsha13057ceeef84dbp102050jsn189f13591088"
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}
    headers = {
        'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
        'x-rapidapi-key': flight_api_key
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    arrivals_airport = response.json()['arrivals']
    arrivals_airport = pd.DataFrame([get_flight_info(flight, airport_icoa) for flight in arrivals_airport])
#Get weather information
    city = "Stuttgart"
    country = "DE"
    OWM_key = "0c87854d706efc6d2ef016fb03918d7a"
    response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
    forecast_api = response.json()['list']
    weather_info = []
#Using for loop generate and inser the weather information inside of the Pandas DataFrame 
#datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
    for forecast_3h in forecast_api: 
        weather_hour = {}
    # datetime utc
        weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
        weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
        weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
        try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
        except: weather_hour['prob_perc'] = 0
    # rain
        try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
        except: weather_hour['rain_qty'] = 0
    # wind 
        try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
        except: weather_hour['snow'] = 0
        weather_hour['municipality_iso_country'] = city + ',' + country
        weather_info.append(weather_hour)
#Creating the DataFrame for weather
    weather_data = pd.DataFrame(weather_info)
#Creating the Dataframe for population
    list_of_city_info = []
    for city in cities:
        url = 'https://en.wikipedia.org/wiki/{}'.format(city)
        web = requests.get(url,'html.parser')
        soup = bs(web.content)
        list_of_city_info.append(City_info(soup))
    df_cities = pd.DataFrame(list_of_city_info)
#Insert credentials from the MySQL Workbench connection 
    schema="test_1"
    host="wbs-project4-gans.c0gfsptlmf5s.eu-central-1.rds.amazonaws.com"
# wbs-project3-db.cfeeyohoevlp.eu-west-2.rds.amazonaws.com
    user="admin"
    password="Akbar3171418*"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
# insert data to tables
    df_cities.to_sql('cities', if_exists='append', con=con, index=False)
    (weather_data
     .assign(datetime = lambda x: pd.to_datetime(x['datetime']))
     .to_sql('weather', if_exists='append', con=con, index=False)
    )
# flight arrivals
    (arrivals_airport
     .replace({np.nan},'unknown')
     .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
     .to_sql('arrivals', if_exists='append', con=con, index=False))
     .to_sql('arrivals2', if_exists='append', con=con, index=False))